In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
from sklearn.datasets import load_sample_image
from sklearn.utils import shuffle
from __future__ import print_function
import os
import scipy
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import LSTM
import numpy as np
import random

Using TensorFlow backend.


In [3]:
def recreate_image(codebook, labels, w, h):
    """Recreate the (compressed) image from the code book & labels"""
    d = codebook.shape[1]
    image = np.zeros((w, h, d))
    label_idx = 0
    for i in range(w):
        for j in range(h):
            image[i][j] = codebook[labels[label_idx]]
            label_idx += 1
    return image

def quantize(image, n_colors):
    image = np.array(image, dtype=np.float64) / 255

    w, h, d = tuple(image.shape)
    assert d == 3
    image_array = np.reshape(image, (w * h, d))

    image_array_sample = shuffle(image_array, random_state=0)[:1000]
    kmeans = KMeans(n_clusters=n_colors, random_state=0).fit(image_array_sample)
    labels = kmeans.predict(image_array)

    return recreate_image(kmeans.cluster_centers_, labels, w, h)


In [4]:
def rgb_to_hex(array):
    array = map(lambda x: int(x * 255), array)
    return '{:02x}{:02x}{:02x}'.format(*array)

def hex_to_rgb(*array):
    rgb_array = []
    for value in array:
        value = value.lstrip('#')
        lv = len(value)
        rgb_value = tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))
        rgb_array.append(rgb_value)
    rgb_array = np.array(rgb_array)
    return rgb_array

h2rgb = np.vectorize(hex_to_rgb)

def hex_image_to_rgb(image_data):
    image_data = image_data.tolist()
    for i, row in enumerate(image_data):
        image_data[i] = h2rgb(image_data[i])[0]
    return np.array(image_data)

In [5]:
input_folder = '../data/monet'
image_width = image_height = 75
input_layer_size = image_width*image_height
input_images = os.listdir(input_folder)
n_samples = len(input_images)
n_colors = 1000

In [6]:
text = np.empty((0,0))
text = []

In [7]:
print('Reading and Quantizing input images')
for input_image in input_images:
    print('loading image {}'.format(input_image))
    image_data = scipy.misc.imread(os.path.join(input_folder,input_image))
    image_data = scipy.misc.imresize(image_data, (image_height, image_width))
    image_data = image_data.reshape(input_layer_size,3)
    text.append(image_data)

Reading and Quantizing input images
loading image images (5).jpg
loading image images (23).jpg
loading image download (4).jpg
loading image download (5).jpg
loading image images (26).jpg
loading image images (30).jpg
loading image images (8).jpg
loading image images (16).jpg
loading image images (10).jpg
loading image images (24).jpg
loading image images (22).jpg
loading image images (37).jpg
loading image images (38).jpg
loading image images (25).jpg
loading image images (32).jpg
loading image images (34).jpg
loading image images (15).jpg
loading image images (21).jpg
loading image images (35).jpg
loading image download.jpg
loading image images (11).jpg
loading image download (2).jpg
loading image images (18).jpg
loading image images (28).jpg
loading image download (1).jpg
loading image images (36).jpg
loading image images (9).jpg
loading image images (20).jpg
loading image download (8).jpg
loading image images (4).jpg
loading image images (14).jpg
loading image download (7).jpg
loadi

In [8]:
text = np.array(text)
text = quantize(text, n_colors)
text = np.apply_along_axis(rgb_to_hex, 2, text)
text = text.reshape(n_samples*input_layer_size)


In [9]:
chars = np.unique(text)
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


total chars: 1000


In [11]:
maxlen = image_width
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 91850


In [12]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype='uint8')
y = np.zeros((len(sentences), len(chars)), dtype='uint8')
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1.
    y[i, char_indices[next_chars[i]]] = 1.

Vectorization...


In [19]:
print('Build model...')
model = Sequential()
model.add(LSTM(512, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 75, 512)           3098624   
_________________________________________________________________
dropout_2 (Dropout)          (None, 75, 512)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              513000    
_________________________________________________________________
activation_1 (Activation)    (None, 1000)              0         
Total params: 5,710,824
Trainable params: 5,710,824
Non-trainable params: 0
___________________________________________________

In [20]:
def sample(a, diversity=0.75):
    if random.random() > diversity:
        return np.argmax(a)
    while 1:
        i = random.randint(0, len(a)-1)
        if a[i] > random.random():
            return i

In [21]:
main_iteration = 0
while 1:
    main_iteration +=1
    print()
    print('-' * 50)
    print('Iteration', main_iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.4, 0.6, 0.8]:
        print()
        print('----- diversity:', diversity)

        generated = []
        sentence = text[start_index : start_index + maxlen]
        generated.extend(sentence)
        pred_image = []
        for iteration in range(input_layer_size):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence  = np.append(sentence[1:], next_char) 
            pred_image.append(next_char)
        pred_image = np.array(pred_image)
        pred_image = hex_image_to_rgb(pred_image)
        pred_image = pred_image.reshape(image_width, image_height, 3) 
        pred_image_filename =  'output/{}_{}.png'.format(main_iteration, diversity)
        print('Saving output image as {}'.format(pred_image_filename))
        scipy.misc.imsave(pred_image_filename, pred_image)



--------------------------------------------------
Iteration 1


/usr/local/lib/python2.7/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
91850/91850 [==============================] - 385s - loss: 6.8007      

IOError: [Errno 2] No such file or directory: 'output/1_0.2.png'